# Analysis prompt 1

"What factors best predict whether an individual is categorized as in permanent housing or not in permanent housing"?


## Purpose

The purpose of this notebook is to give an indication about which factors contribute to a succesful leave out of homelessness. 

### Requirements

A DataFrame will be created that contains several attributes about an individual. 
This means that the DataFrame is aggregated at the individual level.

- Author: Annalie Kruseman
- Date: 2016-10-31

In [1]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
# %load_ext autoreload
# # the "1" means: always reload modules marked with "%aimport"
# %autoreload 1

from __future__ import absolute_import, division, print_function
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import os, sys
# from tqdm import tqdm
# import warnings

sns.set_context("poster", font_scale=1.0)

In [2]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [3]:
# where the data is stored

matt = 'SampleDataSet-MOSBE&SCz-2012-1001--2016-0531/USE THIS ONE/'
annalie = 'data/c4sf/DATA_CTA/'

datadir = os.path.join(os.getenv('HOME'), annalie)

In [4]:
def encode_boolean(df, col):
    '''Encode values as booleans.
    If the string is 'Yes', the new value will be True. Otherwise it will be False.
    '''
    df.loc[df[col] == 'Yes', col] = True
    df.loc[df[col] == 'No', col] = False
    df.loc[df[col] == 'Not Applicable - Child', col] = False
    df.loc[df[col].isin(['Client refused',
                         "Client doesn't know",
                         'Data not collected',
                         '',
                         np.nan]), col] = False
    return df

In [5]:
def encode_unknown(df, col):
    '''Change non-informative values to 'Unknown'.
    '''
    df.loc[df[col].isin(['Client refused',
                         'Refused',
                         "Client doesn't know",
                         'Data not collected']), col] = 'Unknown'
    return df

In [6]:
sheet = 'Client'

cols = [
    'Personal ID',
    'Race',
    'Ethnicity',
    'Gender',
    'Veteran Status',
    ]

infile = os.path.join(datadir, '{s}.csv'.format(s=sheet))

df_client = pd.read_csv(infile, header=0, index_col=0, usecols=cols, sep=';')
df_client = df_client.dropna(how='all')
df_client.index = df_client.index.astype('int')

# drop people that we don't have demographic information for
df_client = df_client.dropna(how='any', subset=['Race', 'Ethnicity', 'Gender'])

# fill in missing values
df_client['Veteran Status'] = df_client['Veteran Status'].fillna(value='')

# Remove "(HUD) from strings
cols = ['Race', 'Ethnicity', 'Veteran Status']
for col in cols:
    df_client[col] = df_client[col].apply(lambda x: x.replace(' (HUD)', ''))

# put the nans back
df_client.loc[df_client['Veteran Status'] == '', 'Veteran Status'] = np.nan

# and encode booleans
df_client = encode_boolean(df_client, 'Veteran Status')

# Some are unknown
df_client = encode_unknown(df_client, 'Race')
df_client = encode_unknown(df_client, 'Ethnicity')
df_client = encode_unknown(df_client, 'Gender')

In [7]:
sheet = 'Enrollment'

cols = [
    'Personal ID',
    'Project Entry ID',
    'Client Age at Entry',
    'Last Permanent Zip',
    'Entry Date',
    'Exit Date',
    'Project ID',
    'Housing Status @ Project Start',
    'Living situation before program entry?',
    'Client Location',
    'Household ID',
    'Relationship to HoH',
    'Disabling Condition',
    'Continuously Homeless One Year',
    'Times Homeless Past Three Years',
    'Months Homeless This Time',
    'Chronic Homeless',
    'In Permanent Housing',
    'Residential Move In Date',
    'Domestic Violence Victim',
    'DV When Occurred',
    'DV Currently Fleeing',
    ]

infile = os.path.join(datadir, '{s}.csv'.format(s=sheet))

df_enroll = pd.read_csv(infile, header=0, index_col=0, usecols=cols, sep=';',
                        parse_dates=['Entry Date', 'Exit Date', 'Residential Move In Date'],
                        infer_datetime_format=True)

df_enroll = df_enroll.dropna(axis=0, how='all')
df_enroll.index = df_enroll.index.astype('int')

# drop anyone for whom we don't have age
df_enroll = df_enroll.dropna(subset=['Client Age at Entry'])

# turn these into integers
cols = ['Project Entry ID', 'Client Age at Entry', 'Project ID', 'Household ID']
for col in cols:
    df_enroll[col] = df_enroll[col].astype('int')

# Remove "(HUD) from strings
cols = ['Housing Status @ Project Start',
        'Living situation before program entry?',
        'Disabling Condition',
        'Continuously Homeless One Year',
        'Domestic Violence Victim',
        'DV When Occurred',
        'DV Currently Fleeing',
        ]
for col in cols:
    df_enroll[col] = df_enroll[col].fillna(value='')
    df_enroll[col] = df_enroll[col].apply(lambda x: x.replace(' (HUD)', ''))
    # put the nans back
    df_enroll.loc[df_enroll[col] == '', col] = np.nan

# encode booleans

# cols = [
#     'Disabling Condition',
#     'Continuously Homeless One Year',
#     'Chronic Homeless',
#     'In Permanent Housing',
#     'Domestic Violence Victim',
#     'DV Currently Fleeing', ]

col = 'Disabling Condition'
df_enroll = encode_boolean(df_enroll, col)

col = 'Continuously Homeless One Year'
df_enroll = encode_boolean(df_enroll, col)

col = 'Chronic Homeless'
df_enroll = encode_boolean(df_enroll, col)

col = 'In Permanent Housing'
df_enroll = encode_boolean(df_enroll, col)

col = 'Domestic Violence Victim'
df_enroll = encode_boolean(df_enroll, col)

col = 'DV Currently Fleeing'
df_enroll = encode_boolean(df_enroll, col)

In [8]:
# one person has a negative age. make it positive.
col = 'Client Age at Entry'
df_enroll.loc[df_enroll[col] < 0, col] = df_enroll.loc[df_enroll[col] < 0, col] * -1

In [9]:
sheet = 'Disability'

cols = [
    'Personal ID',
    'Disability Type',
    'Receiving Services For',
    'Disabilities ID',
    'Project Entry ID',
    ]

infile = os.path.join(datadir, '{s}.csv'.format(s=sheet))

df_disability = pd.read_csv(infile, header=0, index_col=0, usecols=cols, sep=',')
df_disability = df_disability.dropna(axis=0, how='all')
# df_disability.index = df_disability.index.astype('int') # TypeError: Setting dtype to anything other than float64 or object is not supported

# turn these into integers
cols = ['Disabilities ID', 'Project Entry ID']
for col in cols:
    df_disability[col] = df_disability[col].astype('int')

# Remove "(HUD) from strings
cols = ['Disability Type',
        'Receiving Services For',
        ]
for col in cols:
    df_disability[col] = df_disability[col].fillna(value='')
    df_disability[col] = df_disability[col].apply(lambda x: x.replace(' (HUD)', ''))
    # put the nans back
    df_disability.loc[df_disability[col] == '', col] = np.nan

# encode booleans
col = 'Receiving Services For'
df_disability = encode_boolean(df_disability, col)

In [10]:
sheet = 'HealthInsurance'

cols = [
    'Personal ID',
    'Health Insurance Information Date',
    'Health Insurance',
    'Data Collection Stage',
    ]

infile = os.path.join(datadir, '{s}.csv'.format(s=sheet))

df_healthins = pd.read_csv(infile, header=0, index_col=0, usecols=cols,
                           parse_dates=['Health Insurance Information Date'],
                           infer_datetime_format=True)

df_healthins = df_healthins.dropna(axis=0, how='all')
# df_healthins.index = df_healthins.index.astype('int')

In [11]:
sheet = 'Benefit'

cols = [
    'Personal ID',
    'Non-Cash Benefit',
    'Data Collection Stage',
    ]

infile = os.path.join(datadir, '{s}.csv'.format(s=sheet))

df_benefit = pd.read_csv(infile, header=0, index_col=0, usecols=cols, sep=';')

df_benefit = df_benefit.dropna(axis=0, how='all')
df_benefit.index = df_benefit.index.astype('int')

# Drop any project missing the code
df_benefit = df_benefit.dropna(how='any', subset=['Non-Cash Benefit'])

# Remove "(HUD) from strings
cols = ['Non-Cash Benefit',
        ]
for col in cols:
    df_benefit[col] = df_benefit[col].fillna(value='')
    df_benefit[col] = df_benefit[col].apply(lambda x: x.replace(' (HUD)', ''))
    # put the nans back
    df_benefit.loc[df_benefit[col] == '', col] = np.nan

# shorten some column values
col = 'Non-Cash Benefit'
df_benefit.loc[df_benefit[col] == 'Supplemental Nutrition Assistance Program (Food Stamps)', col] = 'Food Stamps'
df_benefit.loc[df_benefit[col] == 'Special Supplemental Nutrition Program for WIC', col] = 'WIC'
df_benefit.loc[df_benefit[col] == 'Section 8, Public Housing, or other ongoing rental assistance', col] = 'Section 8, Public Housing'

In [12]:
sheet = 'Income Entry & Exit'

cols = [
    'Personal ID',
    'Project Entry ID',
    'Entry Alimony',
    'Entry Child Support',
    'Entry Earned',
    'Entry GA',
    'Entry Other',
    'Entry Pension',
    'Entry Private Disability',
    'Entry Social Security Retirement',
    'Entry SSDI',
    'Entry SSI',
    'Entry TANF',
    'Entry Unemployment',
    'Entry VA Non-Service',
    'Entry VA Service Connected',
    "Entry Worker's Compensation",
    'Entry Total Income',
    'Exit Alimony',
    'Exit Child Support',
    'Exit Earned',
    'Exit GA',
    'Exit Other',
    'Exit Pension',
    'Exit Private Disability',
    'Exit Social Security Retirement',
    'Exit SSDI',
    'Exit SSI',
    'Exit TANF',
    'Exit Unemployment',
    'Exit VA Non-Service',
    'Exit VA Service Connected',
    "Exit Worker's Compensation",
    'Exit Total Income',
    'Income Change',
    ]

infile = os.path.join(datadir, '{s}.csv'.format(s=sheet))

df_income = pd.read_csv(infile, header=0, index_col=0, usecols=cols)

df_income = df_income.dropna(axis=0, how='all')
# df_income.index = df_income.index.astype('int')

# turn these into integers
cols = ['Project Entry ID']
for col in cols:
    df_income[col] = df_income[col].astype('int')

# assume all nans are $0
df_income = df_income.fillna(value='0')

# turn the dollar strings into integers
for col in df_income.columns:
    if col != 'Project Entry ID':
        df_income[col] = df_income[col].str.replace(',', '')
        df_income[col] = df_income[col].str.replace(r'[^-+\d.]', '').astype(int)

In [13]:
sheet = 'Service'

cols =  [
    'Personal ID',
    'Services ID',
    'Date Provided',
    'Date Ended',
    'Service Code',
    'Description',
    'Project ID',
    'Record Type',
    'Project Entry ID',
    ]

infile = os.path.join(datadir, '{s}.csv'.format(s=sheet))

df_service = pd.read_csv(infile, header=0, index_col=0, usecols=cols, sep=';',
                         parse_dates=['Date Provided', 'Date Ended'],
                         infer_datetime_format=True)

df_service = df_service.dropna(axis=0, how='all')
df_service.index = df_service.index.astype('int')

# Drop anyone missing these IDs
df_service = df_service.dropna(how='any', subset=['Project ID', 'Project Entry ID'])

# turn these into integers
cols = ['Project ID', 'Project Entry ID']
for col in cols:
    df_service[col] = df_service[col].astype('int')

In [14]:
# drop any null dates
df_service = df_service.dropna(subset=['Date Provided', 'Date Ended'])

In [15]:
sheet = 'Project'

cols = [
    'Project ID',
    'Project Name',
    'Project Type Code',
    'Address City',
    'Organization Name',
    'Address Postal Code'
    ]

infile = os.path.join(datadir, '{s}.csv'.format(s=sheet))

df_project = pd.read_csv(infile, header=0, index_col=1, usecols=cols, sep=';')

df_project.head()

df_project = df_project.dropna(axis=0, how='all')
df_project.index = df_project.index.astype('int')

# Drop any project missing the code
df_project = df_project.dropna(how='any', subset=['Project Type Code'])

# Remove "(HUD) from strings
cols = ['Project Type Code',
        ]
for col in cols:
    df_project[col] = df_project[col].fillna(value='')
    df_project[col] = df_project[col].apply(lambda x: x.replace(' (HUD)', ''))
    # put the nans back
    df_project.loc[df_project[col] == '', col] = np.nan

In [16]:
sheet = 'BedInventory'

cols = [
    'Project ID',
    'Inventory ID',
    'Inventory Household Type',
    'HMIS Participating Beds',
    'Inventory Start Date',
    'Inventory End Date',
    'Unit Inventory',
    'Bed Inventory',
    'Vet Bed Inventory',
    'Youth Bed Inventory',
    'Youth Bed Age Group',
    ]

infile = os.path.join(datadir, '{s}.csv'.format(s=sheet))

df_bedinv = pd.read_csv(infile, header=0, index_col=0, usecols=cols,
                        parse_dates=['Inventory Start Date', 'Inventory End Date'],
                        infer_datetime_format=True)

df_bedinv = df_bedinv.dropna(axis=0, how='all')
# df_bedinv.index = df_bedinv.index.astype('int')

# turn these into integers, assume zero if NaN
cols = ['Inventory ID', 'HMIS Participating Beds', 'Unit Inventory', 'Bed Inventory', 'Vet Bed Inventory', 'Youth Bed Inventory']
for col in cols:
    # df_bedinv[col] = df_bedinv.loc[~df_bedinv[col].isnull(), col].apply(lambda x: int(x))
    df_bedinv[col] = df_bedinv[col].fillna(value=0)
    df_bedinv[col] = df_bedinv[col].astype('int')

# Clean data and make some features

In [17]:
# Only keep rows with entry dates starting in 2011
df_enroll = df_enroll[df_enroll['Entry Date'] >= '2011']

# Only keep rows with exit date
df_enroll = df_enroll[df_enroll['Exit Date'].notnull()]

In [18]:
# calculate the number of days that someone was enrolled
df_enroll['Days Enrolled'] = ((df_enroll['Exit Date'] - df_enroll['Entry Date']) / np.timedelta64(1, 'D')).astype(int)

In [19]:
# remove anyone with negative number of enrollment days
df_enroll = df_enroll[df_enroll['Days Enrolled'] >= 0]

In [20]:
# Join the client information with enrollment information.
# Inner join because we need both.
df = df_client.merge(df_enroll, how='inner', left_index=True, right_index=True)

In [21]:
# Join the merged new dataframe with benefit information.
df = df.merge(df_benefit.reset_index().groupby(by=['Personal ID'])[['Non-Cash Benefit']].nth(0),
              how='left', left_index=True, right_index=True)

df['Non-Cash Benefit'] = df['Non-Cash Benefit'].fillna('None')

In [22]:
# Join the merged new dataframe with disability information.
df = df.merge(df_disability.reset_index().groupby(by=['Personal ID'])[['Disability Type']].nth(0),
              how='left', left_index=True, right_index=True)

df['Disability Type'] = df['Disability Type'].fillna('None')

In [23]:
# merge organizations from the project dataframe with the merged client and enrollment dataframe

# make column of index from df dataframe (Personal ID)
df_resetindex = df.reset_index()
# make column of index from project dataframe (Project ID)
df_project_resetindex = df_project.reset_index()

# merge new df dataframe with project dataframe on Project ID
df_dfProject = df_resetindex.merge(df_project_resetindex, how='left', on='Project ID').set_index('Personal ID').sort_index()

In [24]:
# when 'Days Enrolled' is 0 and 'Non-Cash Benefit' is foodstamp, rename 0 value to 1
df_dfProject.ix[(df_dfProject['Days Enrolled'] == 0) & (df_dfProject['Non-Cash Benefit'] == 'Food Stamps'), 'Days Enrolled'] = 1

In [25]:
df_dfProject['Non-Cash Benefit'].value_counts()

None                            39890
Food Stamps                     20938
Other Source                      992
Section 8, Public Housing         964
WIC                               611
Other TANF-Funded Services        183
TANF Transportation Services       42
TANF Child Care Services           33
Temporary rental assistance        13
Name: Non-Cash Benefit, dtype: int64

    NB: What does it mean that someone doesn't receive a 'Non-Cash Benefit' (value = None), but for example, is
    registered for Emergency Shelter?

In [26]:
# remove all 'None' values for 'Non-Cash Benefit'
df_dfProject = df_dfProject[df_dfProject['Non-Cash Benefit'] != 'None']

In [27]:
df = df_dfProject

In [28]:
# sort by entry date
df = df.sort('Entry Date')

/Users/annalie/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


In [29]:
# set up to count the number of times a person was in the system
df['Enrollments'] = 1

In [197]:
agg = {'In Permanent Housing': 'last',
       'Enrollments': 'sum',
       'Race': 'first',
       'Ethnicity': 'first',
       'Gender': 'first',
       'Veteran Status': 'max',
       'Client Age at Entry': 'last',
       'Days Enrolled': 'sum',
       'Domestic Violence Victim': 'max',
       'Disability Type': 'last',
       'Non-Cash Benefit': 'last',
       'Housing Status @ Project Start': 'last',
       'Living situation before program entry?': 'last',
       'Continuously Homeless One Year': 'max',
#        'Months Homeless This Time': 'last',
       'Chronic Homeless': 'max',
       'Project Name': 'last',
       'Organization Name': 'last',
       'Project Type Code': 'last',
#        'Address City': 'last',
#        'Address Postal Code': 'last',
      }
df_features = df.reset_index().groupby(by=['Personal ID']).agg(agg)

In [198]:
# replace values in the below columns to values where False = 0 and True = 1
df_features['In Permanent Housing'] = df_features['In Permanent Housing'].map({False: 0, True: 1})
df_features['Chronic Homeless'] = df_features['Chronic Homeless'].map({False: 0, True: 1})
df_features['Domestic Violence Victim'] = df_features['Domestic Violence Victim'].map({False: 0, True: 1})
df_features['Veteran Status'] = df_features['Veteran Status'].map({False: 0, True: 1})
df_features['Continuously Homeless One Year'] = df_features['Continuously Homeless One Year'].map({False: 0, True: 1})

    Remove values with missing values in the column "Months Homeless This Time"

In [32]:
# number of missing values in column 'Months Homeless This Time":
# df_features['Months Homeless This Time'].isnull().sum().sum()

In [58]:
# number of total values
len(df_features)

3397

In [157]:
df_features.head(1)

,Non-Cash Benefit,Chronic Homeless,Domestic Violence Victim,Housing Status @ Project Start,Gender,Veteran Status,Days Enrolled,Disability Type,Living situation before program entry?,In Permanent Housing,Race,Client Age at Entry,Project Name,Enrollments,Organization Name,Project Type Code,Ethnicity,Continuously Homeless One Year
Personal ID,,,,,,,,,,,,,,,,,,
173781,Food Stamps,0,1,Category 1 - Homeless,Female,0,148,None,"Emergency shelter, including hotel or motel pa...",0,White,35,MOSBE SOP - Natividad Shelter,2,MOSBE SOP,Emergency Shelter,Hispanic/Latino,0


# Build a Logistic Regression model
    (1) Scikit Learn, see below
    (2) Statsmodels, see bottom

# (1) Logistic Regression with Scikit Learn

In [200]:
# create dummy variables for the following variables
for column in ['Non-Cash Benefit', 
               'Housing Status @ Project Start', 
               'Gender', 
               'Disability Type', 
               'Living situation before program entry?', 
               'Race', 
               'Project Name', 
               'Organization Name', 
               'Project Type Code', 
               'Ethnicity']:
    dummies = pd.get_dummies(df_features[column])
    df_features[dummies.columns] = dummies
    
# create a new dataframe that contains only numeric datatypes
df_features_numeric = df_features.drop(['Non-Cash Benefit', 'Housing Status @ Project Start', 'Domestic Violence Victim', 'Gender', 'Disability Type',
                                        'Living situation before program entry?','Race', 'Project Name', 'Organization Name',
                                        'Project Type Code', 'Ethnicity'], axis=1)

df_features_numeric.head(1)

,Chronic Homeless,Veteran Status,Days Enrolled,In Permanent Housing,Client Age at Entry,Enrollments,Continuously Homeless One Year,Food Stamps,Other Source,Other TANF-Funded Services,"Section 8, Public Housing",TANF Child Care Services,TANF Transportation Services,Temporary rental assistance,WIC,At-risk of homelessness,Category 1 - Homeless,Category 2 - At imminent risk of losing housing,Category 3 - Homeless only under other federal statutes,Category 4 - Fleeing domestic violence,Client doesn't know,Client refused,Data not collected,Stably housed,Female,Male,Other,Transgender female to male,Transgender male to female,Unknown,Alcohol Abuse,Both Alcohol and Drug Abuse,Chronic Health Condition,Developmental,Drug Abuse,HIV/AIDS,Mental Health Problem,None,Physical,Substance Abuse,"Emergency shelter, including hotel or motel paid for with emergency shelter voucher(HUD)",Foster care home or foster care group home,Hospital or other residential non-psychiatric medical facility,Hotel or motel paid for without emergency shelter voucher,"Jail, prison or juvenile detention facility","Owned by client, no ongoing housing subsidy","Owned by client, with ongoing housing subsidy",Permanent housing for formerly homeless persons,Place not meant for habitation,Psychiatric hospital or other psychiatric facility,"Rental by client, no ongoing housing subsidy","Rental by client, with VASH subsidy","Rental by client, with other ongoing housing subsidy",Residential project or halfway house with no homeless criteria,Safe Haven,"Staying or living in a family member's room, apartment or house","Staying or living in a friend's room, apartment or house",Substance abuse treatment facility or detox center,Transitional housing for homeless persons (including homeless youth),American Indian or Alaska Native,Asian,Black or African American,Native Hawaiian or Other Pacific Islander,White,MOSBE SOP - Transitional Housing - Men In Transition,MOSBE - Homeless Winter Shelter for Families,MOSBE CHS - Elm House,MOSBE CHS - RHY - BCP - HP,MOSBE CHS - RHY SOP,MOSBE CHS - Safe Passage,MOSBE CHS - Safe Place Warming Shelter,MOSBE CSUMB - Chinatown Project,MOSBE Franciscan Workers - House of Peace,MOSBE Franciscan Workers - Women Alive! Shelter,MOSBE Franciscan Workers of Junipero Serra,MOSBE HRC - LEGACY Emergency Rental Assistance,MOSBE HRC - SSVF- P1 - CCCIL HP,MOSBE HRC - SSVF- P1 - CCCIL RRH,MOSBE HRC - SSVF- P1 - HRC HP,MOSBE HRC - SSVF- P1 - HRC RRH,MOSBE HRC - SSVF- P1 - VTC HP,MOSBE HRC - SSVF- P1 - VTC RRH,MOSBE HRC - SSVF- P2 HRC- RRH - Santa Cruz Clients,MOSBE HRC - SSVF- P2 - HRC RRH,MOSBE HRC - SSVF- P2 HRC - HP- Monterey/San Benito Clients,MOSBE HRC - SSVF- P2 HRC - RRH- Monterey/San Benito Clients,MOSBE HRC - Security Deposit Guarantees,MOSBE Housing Authority - Pueblo del Mar Family Recovery Community,MOSBE Housing Authority - S + C Vouchers,MOSBE Housing Resource Center (HRC)*,MOSBE Interim,MOSBE Interim - MCHOME Enrolled,MOSBE Interim - MCHOME Outreach,MOSBE Interim - MCHOME S+C,MOSBE Interim - MCHOPE,MOSBE Interim - Sandy Shores,MOSBE Interim - Shelter Cove,MOSBE Interim - Sunflower Gardens - PSH,MOSBE Interim - Sunflower Gardens - THU,MOSBE SOP - Hamilton ES,MOSBE SOP - Hamilton Family Units -DO NOT USE-,MOSBE SOP - MLP - Emergency Men's Program,MOSBE SOP - Mobile Outreach Shelter Program - MOST,MOSBE SOP - Natividad Shelter,MOSBE SOP - Transitional Housing - Lexington Court,MOSBE SOP - Transitional Housing - Wittenmyer (Homeward Bound),MOSBE SOP - Winter Warming Shelter,MOSBE San Benito County - Helping Hands,MOSBE San Benito County - Homeless Warming Shelter,MOSBE The Salvation Army - Casa de las Palmas,MOSBE The Salvation Army - Fredericksen House,MOSBE The Salvation Army - Good Samaritan Center,MOSBE The Salvation Army - Phase II,MOSBE VTC DO NOT ENTER HERE,MOSBE Veterans Transition Center - GPD Housing,MOSBE Veterans Transition Center - Outreach,SCz - CAB TSP Rental Assistance,SCz - County Office of Education,SCz - Encompass,SCz - Encompass - Ander

# Create dataframes with dummie variables for the variables seperately

In [94]:
## NOT NECCESSARY FOR STATSMODELS

# create dataframe for only 'Non Cash Benefit' with dummy variables
df_features_non_cash_benefit = df_features[['In Permanent Housing', 'Non-Cash Benefit']]

# create dummy variables for the following variables
dummies_non_cash_benefit = pd.get_dummies(df_features_non_cash_benefit['Non-Cash Benefit'])
df_features_non_cash_benefit[dummies_non_cash_benefit.columns] = dummies_non_cash_benefit

# create a new dataframe that contains only numeric datatypes
df_features_non_cash_benefit = df_features_non_cash_benefit.drop('Non-Cash Benefit', axis=1)
df_features_non_cash_benefit.head()

,In Permanent Housing,Food Stamps,Other Source,Other TANF-Funded Services,"Section 8, Public Housing",TANF Child Care Services,TANF Transportation Services,Temporary rental assistance,WIC
Personal ID,,,,,,,,,
173781,0,1,0,0,0,0,0,0,0
173803,0,1,0,0,0,0,0,0,0
173848,0,1,0,0,0,0,0,0,0
173885,0,1,0,0,0,0,0,0,0
173899,0,1,0,0,0,0,0,0,0


In [95]:
## NOT NECCESSARY FOR STATSMODELS

# create dataframe for only 'Housing Status @ Project Start' with dummy variables
df_features_housing_status = df_features[['In Permanent Housing', 'Housing Status @ Project Start']]

# create dummy variables for the following variables
dummies_housing_status = pd.get_dummies(df_features_housing_status['Housing Status @ Project Start'])
df_features_housing_status[dummies_housing_status.columns] = dummies_housing_status

# create a new dataframe that contains only numeric datatypes
df_features_housing_status = df_features_housing_status.drop('Housing Status @ Project Start', axis=1)
df_features_housing_status.head()

,In Permanent Housing,At-risk of homelessness,Category 1 - Homeless,Category 2 - At imminent risk of losing housing,Category 3 - Homeless only under other federal statutes,Category 4 - Fleeing domestic violence,Client doesn't know,Client refused,Data not collected,Stably housed
Personal ID,,,,,,,,,,
173781,0,0,1,0,0,0,0,0,0,0
173803,0,0,1,0,0,0,0,0,0,0
173848,0,0,1,0,0,0,0,0,0,0
173885,0,0,1,0,0,0,0,0,0,0
173899,0,0,1,0,0,0,0,0,0,0


In [97]:
## NOT NECCESSARY FOR STATSMODELS

# create dataframe for only 'Living situation before program entry?' with dummy variables
df_features_living_situation = df_features[['In Permanent Housing', 'Living situation before program entry?']]

# create dummy variables for the following variables
dummies_living_situation = pd.get_dummies(df_features_living_situation['Living situation before program entry?'])
df_features_living_situation[dummies_living_situation.columns] = dummies_living_situation

# create a new dataframe that contains only numeric datatypes
df_features_living_situation = df_features_living_situation.drop('Living situation before program entry?', axis=1)
df_features_living_situation.head()

,In Permanent Housing,Client doesn't know,Client refused,Data not collected,"Emergency shelter, including hotel or motel paid for with emergency shelter voucher(HUD)",Foster care home or foster care group home,Hospital or other residential non-psychiatric medical facility,Hotel or motel paid for without emergency shelter voucher,"Jail, prison or juvenile detention facility",Other,"Owned by client, no ongoing housing subsidy","Owned by client, with ongoing housing subsidy",Permanent housing for formerly homeless persons,Place not meant for habitation,Psychiatric hospital or other psychiatric facility,"Rental by client, no ongoing housing subsidy","Rental by client, with VASH subsidy","Rental by client, with other ongoing housing subsidy",Residential project or halfway house with no homeless criteria,Safe Haven,"Staying or living in a family member's room, apartment or house","Staying or living in a friend's room, apartment or house",Substance abuse treatment facility or detox center,Transitional housing for homeless persons (including homeless youth)
Personal ID,,,,,,,,,,,,,,,,,,,,,,,,
173781,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
173803,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
173848,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
173885,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
173899,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [98]:
## NOT NECCESSARY FOR STATSMODELS

# create dataframe for only 'Project Name' with dummy variables
df_features_project_name = df_features[['In Permanent Housing', 'Project Name']]

# create dummy variables for the following variables
dummies_project_name = pd.get_dummies(df_features_project_name['Project Name'])
df_features_project_name[dummies_project_name.columns] = dummies_project_name

# create a new dataframe that contains only numeric datatypes
df_features_project_name = df_features_project_name.drop('Project Name', axis=1)
df_features_project_name.head()

,In Permanent Housing,MOSBE SOP - Transitional Housing - Men In Transition,MOSBE - Homeless Winter Shelter for Families,MOSBE CHS - Elm House,MOSBE CHS - RHY - BCP - HP,MOSBE CHS - RHY SOP,MOSBE CHS - Safe Passage,MOSBE CHS - Safe Place Warming Shelter,MOSBE CSUMB - Chinatown Project,MOSBE Franciscan Workers - House of Peace,MOSBE Franciscan Workers - Women Alive! Shelter,MOSBE Franciscan Workers of Junipero Serra,MOSBE HRC - LEGACY Emergency Rental Assistance,MOSBE HRC - SSVF- P1 - CCCIL HP,MOSBE HRC - SSVF- P1 - CCCIL RRH,MOSBE HRC - SSVF- P1 - HRC HP,MOSBE HRC - SSVF- P1 - HRC RRH,MOSBE HRC - SSVF- P1 - VTC HP,MOSBE HRC - SSVF- P1 - VTC RRH,MOSBE HRC - SSVF- P2 HRC- RRH - Santa Cruz Clients,MOSBE HRC - SSVF- P2 - HRC RRH,MOSBE HRC - SSVF- P2 HRC - HP- Monterey/San Benito Clients,MOSBE HRC - SSVF- P2 HRC - RRH- Monterey/San Benito Clients,MOSBE HRC - Security Deposit Guarantees,MOSBE Housing Authority - Pueblo del Mar Family Recovery Community,MOSBE Housing Authority - S + C Vouchers,MOSBE Housing Resource Center (HRC)*,MOSBE Interim,MOSBE Interim - MCHOME Enrolled,MOSBE Interim - MCHOME Outreach,MOSBE Interim - MCHOME S+C,MOSBE Interim - MCHOPE,MOSBE Interim - Sandy Shores,MOSBE Interim - Shelter Cove,MOSBE Interim - Sunflower Gardens - PSH,MOSBE Interim - Sunflower Gardens - THU,MOSBE SOP - Hamilton ES,MOSBE SOP - Hamilton Family Units -DO NOT USE-,MOSBE SOP - MLP - Emergency Men's Program,MOSBE SOP - Mobile Outreach Shelter Program - MOST,MOSBE SOP - Natividad Shelter,MOSBE SOP - Transitional Housing - Lexington Court,MOSBE SOP - Transitional Housing - Wittenmyer (Homeward Bound),MOSBE SOP - Winter Warming Shelter,MOSBE San Benito County - Helping Hands,MOSBE San Benito County - Homeless Warming Shelter,MOSBE The Salvation Army - Casa de las Palmas,MOSBE The Salvation Army - Fredericksen House,MOSBE The Salvation Army - Good Samaritan Center,MOSBE The Salvation Army - Phase II,MOSBE VTC DO NOT ENTER HERE,MOSBE Veterans Transition Center - GPD Housing,MOSBE Veterans Transition Center - Outreach,SCz - CAB TSP Rental Assistance,SCz - County Office of Education,SCz - Encompass,SCz - Encompass - Anderson,SCz - Encompass - River Street Shelter,SCz - Encompass - River Street Shelter PATH,SCz - FIT - Brommer,SCz - FIT - HP SSVF - FIT Clients,SCz - FIT - HP SSVF - HSC Clients,SCz - FIT - HP SSVF - PVSS Clients,SCz - FIT - HUD (Scattered Site),SCz - FIT - HUD/DA (Clean & Sober),SCz - FIT - RRH SSVF - FIT Clients,SCz - FIT - RRH SSVF - HSC Clients,SCz - FIT - RRH SSVF - PVSS Clients,SCz - Front St.,SCz - Front St. - Paget Center,SCz - HPHP-MATCH Housing Inebriates I,SCz - HPHP-MATCH Housing Inebriates III,SCz - HSC - 180/2020,SCz - HSC - NO DATA HERE,SCz - HSC - Page Smith Community House,SCz - HSC - Paul Lee Loft,SCz - HSC - Rebele Family Shelter Program,SCz - HSC - Recuperative Care Center,SCz - HSC - Winter Shelter,SCz - HSD -CHAMP,SCz - HSD -CHAMP- FIT Clients,SCz - PRM - Emergency Shelter,SCz - PVSS - Emergency Shelter,SCz - PVSS - Transitional Housing for Families,SCz - Salvation Army- Winter (Cold Weather) Shelter,SCz-FIT-County Planning Department Grant,SCz-Veterans Resource Center - HP SSVF - Monterey/San Benito Clients,SCz-Veterans Resource Center - RRH SSVF - Monterey/San Benito Clients,SCz-Veterans Resource Center- HP SSVF -San Mateo Clients,SCz-Veterans Resource Center- HP SSVF -Santa Cruz Clients,SCz-Veterans Resource Center- RRH SSVF -San Mateo Clients,SCz-Veterans Resource Center- RRH SSVF -Santa Cruz Clients,SCz-Veterans Resource Center- SSVF - Priority 1 SCz - HP Santa Cruz,SCz-Veterans Resource Center- SSVF - Priority 1 SCz - RRH Santa Cruz,ZZZ* INACTIVE*ZZZ-MOSBE HRC - SSVF-P2 VRSi - Santa Cruz Clients,ZZZ*INACTIVE*ZZZ- MOSBE HRC - SSVF-P2 VRSi- Monterey/San Benito Clients,zzz*Inactive* SCz - CAB - ESG,zzz*Inactive* SCz - FIT - ESG Homeless Prevention,zzz*Inactive* SCz - FIT - ESG Rapid Rehousing,zzz*Inactive* SCz - Salvation Army - Corner House (HUD),zzz*inactive* MOSBE CCHAS (John XXIII) - Voucher Program,zzz*i

In [99]:
## NOT NECCESSARY FOR STATSMODELS

# create dataframe for only 'Project Type Code' with dummy variables
df_features_project_type = df_features[['In Permanent Housing', 'Project Type Code']]

# create dummy variables for the following variables
dummies_project_type = pd.get_dummies(df_features_project_type['Project Type Code'])
df_features_project_type[dummies_project_type.columns] = dummies_project_type

# create a new dataframe that contains only numeric datatypes
df_features_project_type = df_features_project_type.drop('Project Type Code', axis=1)
df_features_project_type.head()

,In Permanent Housing,Emergency Shelter,Homelessness Prevention,Other,PH - Permanent Supportive Housing (disability required for entry),PH - Rapid Re-Housing,Services Only,Street Outreach,Transitional housing
Personal ID,,,,,,,,,
173781,0,1,0,0,0,0,0,0,0
173803,0,1,0,0,0,0,0,0,0
173848,0,0,0,0,0,0,0,0,1
173885,0,0,0,1,0,0,0,0,0
173899,0,1,0,0,0,0,0,0,0


In [100]:
import sklearn
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression

/Users/annalie/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [101]:
# create a feature set and target variable
predictors = df_features_numeric.drop(['In Permanent Housing'], axis=1)
targets = df_features_numeric['In Permanent Housing']

# # Or should I use a list as input?
# predictors_matrix = predictors.as_matrix()
# targets_list = targets.tolist()

In [102]:
# split data set in training and test set
pred_train, pred_test, tar_train, tar_test = train_test_split(predictors, targets, test_size=.3)

# pred_train, pred_test, tar_train, tar_test = train_test_split(predictors_matrix, targets_list, test_size=.3)

In [103]:
# show the shape of the train and test sets
print ("pred_train shape =", pred_train.shape)
print ("pred_test shape =", pred_test.shape)
print ("tar_train shape =", tar_train.shape)
print ("tar_test shape =", tar_test.shape)

pred_train shape = (2377, 198)
pred_test shape = (1020, 198)
tar_train shape = (2377,)
tar_test shape = (1020,)


In [104]:
# fit a logistic regression model to the data
lr_model = LogisticRegression()
lr_model.fit(pred_train, tar_train)
print(lr_model)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


In [105]:
# make predictions
expected = tar_test
predicted = lr_model.predict(pred_test)

In [106]:
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))
print ("accuracy score:", sklearn.metrics.accuracy_score(expected, predicted))

             precision    recall  f1-score   support

          0       0.95      0.98      0.97       917
          1       0.79      0.56      0.66       103

avg / total       0.94      0.94      0.94      1020

[[902  15]
 [ 45  58]]
accuracy score: 0.941176470588


In [ ]:
# plt.scatter(predicted, expected)
# plt.xlabel("predicted probability")
# plt.ylabel("actual outcome")
# plt.title("Logistic REgression Predicted vs. Actual")
# plt.show()

In [70]:
# predict class probabilities of the input samples
predict_probability_lr = lr_model.predict_proba(pred_test)
# list(predict_probability_lr)

In [71]:
# list(predicted)

In [72]:
# get coefficient per predictor

# 1. create list of coefficients and predictors
coef = lr_model.coef_
coef = coef.tolist()
# unnest nested list
coef = sum(coef, [])

predictors_columns = list(predictors.columns)

In [73]:
def mean(x):
    return sum(x) / len(x)

# standard error (deviation from the mean)
def de_mean(x):
    x_bar = mean(x)
    return [x_i - x_bar for x_i in x]

In [74]:
mean(de_mean(predictors['Chronic Homeless']))

2.0007382399172953e-16

In [75]:
st_error = []
for i in pred_test.columns:
    st_error.append(mean(de_mean(predictors[i])))

In [76]:
predictors_coef_sterror = zip(predictors_columns, coef, st_error)

In [77]:
predictors_coef_sterror_matrix = np.array(predictors_coef_sterror)
predictors_coef_sterror_matrix[1]

predictors_coef_sterror_matrix_T = predictors_coef_sterror_matrix.transpose()
predictors = predictors_coef_sterror_matrix_T[0]
coef = predictors_coef_sterror_matrix_T[1].astype(float)
st_error = predictors_coef_sterror_matrix_T[2].astype(float)

print (predictors[:5])
print (coef[:5])
print (st_error[:5])

['Chronic Homeless' 'Veteran Status' 'Days Enrolled' 'Client Age at Entry'
 'Enrollments']
[ -5.47986270e-01   9.14466229e-01   6.95240477e-04  -6.27860434e-03
   3.48483870e-03]
[  2.00073824e-16  -1.46384720e-15   3.21539308e-12  -3.44624731e-14
   5.73935300e-14]


In [78]:
import numpy as np
predictors_coef_sterror_matrix_T = predictors_coef_sterror_matrix.transpose()
z_score = np.divide(coef, st_error)
z_score[:5]

array([ -2.73892036e+15,  -6.24700604e+14,   2.16222545e+08,
         1.82186703e+11,   6.07183197e+10])

In [79]:
statistics = np.column_stack((predictors, coef, st_error, z_score))
statistics[0]

array(['Chronic Homeless', '-0.547986269831', '2.00073823992e-16',
       '-2.73892035898e+15'], 
      dtype='|S88')

In [80]:
# sort predictors by highest to lowest z-score
def getKey(item):
    return item[3]

statistics_sorted = sorted(statistics, key = getKey, reverse = True)

In [81]:
columns = ['predictor', 'coefficient', 'standard error', 'z-score']

output_lr = pd.DataFrame(data = statistics_sorted, columns = columns)
output_lr.head()

,predictor,coefficient,standard error,z-score
0,MOSBE SOP,-0.408404158005,-4.13768061452e-16,9.87036448806e+14
1,MOSBE Interim - MCHOME Enrolled,0.695917571588,7.09005026213e-18,9.81541097536e+16
2,PH - Rapid Re-Housing,1.89441707344,2.00422709208e-15,9.45210790197e+14
3,"Rental by client, with other ongoing housing s...",0.831438349673,8.83406781149e-17,9.41172704823e+15
4,MOSBE - Homeless Winter Shelter for Families,-0.208447967894,-2.21791315892e-17,9.39838275704e+15


# Logistic regression with cross-validation

In [ ]:
## CROSS-VALIDATION DOES NOT IMPROVE THE PREDICTION

# cross_val_score automatically equals the size of the training and test set
from sklearn.cross_validation import cross_val_score

# perform 10-fold cross validation
lr_model_crossval = LogisticRegression()

scores = cross_val_score(lr_model_crossval, predictors, targets, cv = 10, scoring = 'accuracy')
print ('scores for all the 10 samples: ')
print (scores)
print ("---")
print ('mean score:')
print (scores.mean())

# run a logistic regression for each variable seperately

In [ ]:
# create a feature set and target variable
predictors_housing_status = df_features_numeric[['Category 1 - Homeless',
                                                 'Category 2 - At imminent risk of losing housing',
                                                'Category 3 - Homeless only under other federal statutes',
                                                'Category 4 - Fleeing domestic violence']]
targets = df_features_numeric['In Permanent Housing']

result.summary()

logit = sm.Logit(targets, predictors_housing_status)
result = logit.fit()

result.pred_table()

# get the marginal effect
marginal_effect_x = result.get_margeff()
print (marginal_effect_x.summary())

# (2) Logistic Regression with Statsmodels

In [119]:
import statsmodels.api as sm
from statsmodels.formula.api import logit, probit, poisson, ols
import sklearn
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression

In [120]:
import statsmodels.formula.api as smf 

# Make logistic regressions for each variable on 'In Permanent Housing'

In [190]:
# remove spaces in the variables 
df_features = df_features.rename(columns={'In Permanent Housing': 'in_permanent_housing', 
                                          'Non-Cash Benefit': 'non_cash_benefit',
                                          'Chronic Homeless': 'chronic_homeless',
                                          'Domestic Violence Victim': 'domestic_violence_victim',
                                          'Housing Status @ Project Start': 'housing_status_project_start',
                                          'Gender': 'gender',
                                          'Veteran Status': 'veteran_status',
                                          'Days Enrolled': 'days_enrolled',
                                          'Disability Type': 'disability_type',
                                          'Living situation before program entry?': 'living_situation_before_program_entry',
                                          'Race': 'race',
                                          'Client Age at Entry': 'client_age_at_entry',
                                          'Project Name': 'project_name',
                                          'Enrollments': 'enrollments',
                                          'Organization Name': 'organization_name',
                                          'Project Type Code': 'project_type_code',
                                          'Ethnicity': 'ethnicity',
                                          'Continuously Homeless One Year': 'continuously_homeless_one_year'
                                          })
df_features.head(1)

,non_cash_benefit,chronic_homeless,domestic_violence_victim,housing_status_project_start,gender,veteran_status,days_enrolled,disability_type,living_situation_before_program_entry,in_permanent_housing,race,client_age_at_entry,project_name,enrollments,organization_name,project_type_code,ethnicity,continuously_homeless_one_year
Personal ID,,,,,,,,,,,,,,,,,,
173781,Food Stamps,0,1,Category 1 - Homeless,Female,0,148,None,"Emergency shelter, including hotel or motel pa...",0,White,35,MOSBE SOP - Natividad Shelter,2,MOSBE SOP,Emergency Shelter,Hispanic/Latino,0


In [187]:
# # logistic regression for Non Cash Benefit

logit_benefit = smf.logit(formula='in_permanent_housing ~ C(non_cash_benefit)', 
                          data=df_features).fit()
# odds ratios
print("Odds Ratios")

print(np.exp(logit_benefit.params))
print(logit_benefit.summary())

         Current function value: 0.313057
         Iterations: 35
Odds Ratios
Intercept                                              9.909570e-02
C(non_cash_benefit)[T.Other Source]                    1.486309e+00
C(non_cash_benefit)[T.Other TANF-Funded Services]      2.632501e+00
C(non_cash_benefit)[T.Section 8, Public Housing]       8.199144e+00
C(non_cash_benefit)[T.TANF Child Care Services]        1.379957e-10
C(non_cash_benefit)[T.TANF Transportation Services]    1.008826e-09
C(non_cash_benefit)[T.Temporary rental assistance]     1.018487e-07
C(non_cash_benefit)[T.WIC]                             4.740858e-01
dtype: float64
                            Logit Regression Results                            
Dep. Variable:     in_permanent_housing   No. Observations:                 3397
Model:                            Logit   Df Residuals:                     3389
Method:                             MLE   Df Model:                            7
Date:                  Tue, 15 Nov 2016

/Users/annalie/anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [168]:
# Logistic Regression for 'Housing Status @ Project Start'

# rename variable
logit_housing_status = smf.logit(formula='in_permanent_housing ~ C(housing_status_project_start)', 
                                 data=df_features).fit()
# odds ratios
print("Odds Ratios")
print(np.exp(logit_housing_status.params))
print(logit_housing_status.summary())

         Current function value: 0.284540
         Iterations: 35
Odds Ratios
Intercept                                                                                     2.571428e-01
C(housing_status_project_start)[T.Category 1 - Homeless]                                      3.060778e-01
C(housing_status_project_start)[T.Category 2 - At imminent risk of losing housing]            3.431373e-01
C(housing_status_project_start)[T.Category 3 - Homeless only under other federal statutes]    1.013427e-08
C(housing_status_project_start)[T.Category 4 - Fleeing domestic violence]                     1.013427e-08
C(housing_status_project_start)[T.Client doesn't know]                                        1.944445e-01
C(housing_status_project_start)[T.Client refused]                                             9.798717e-19
C(housing_status_project_start)[T.Data not collected]                                         8.907272e-07
C(housing_status_project_start)[T.Stably housed]                  

/Users/annalie/anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [169]:
# Logistic Regression for 'Living situation before program entry?'

# rename variable
logit_living_situation = smf.logit(formula='in_permanent_housing ~ C(living_situation_before_program_entry)', 
                                   data=df_features).fit()
# odds ratios
print("Odds Ratios")
print(np.exp(logit_living_situation.params))
print(logit_living_situation.summary())

         Current function value: 0.304461
         Iterations: 35
Odds Ratios
Intercept                                                                                                                               5.099716e-13
C(living_situation_before_program_entry)[T.Client refused]                                                                              6.703268e+02
C(living_situation_before_program_entry)[T.Data not collected]                                                                          3.352223e+04
C(living_situation_before_program_entry)[T.Emergency shelter, including hotel or motel paid for with emergency shelter voucher(HUD)]    7.873568e+10
C(living_situation_before_program_entry)[T.Foster care home or foster care group home]                                                  2.625866e+04
C(living_situation_before_program_entry)[T.Hospital or other residential non-psychiatric medical facility]                              1.668845e+11
C(living_situation_before_pr

/Users/annalie/anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [ ]:
## too many iterations, > 35

# Logistic Regression for 'Project Name'

# rename variable
logit_project_name = smf.logit(formula='in_permanent_housing ~ C(project_name)', 
                               data=df_features).fit()
# odds ratios
print("Odds Ratios")
print(np.exp(logit_project_name.params))
print(logit_project_name.summary())

In [171]:
# Logistic Regression for 'Project Type Code'

# rename variable
logit_project_type = smf.logit(formula='in_permanent_housing ~ C(project_type_code)', 
                               data=df_features).fit()
# odds ratios
print("Odds Ratios")
print(np.exp(logit_project_type.params))
print(logit_project_type.summary())

Optimization terminated successfully.
         Current function value: 0.222571
         Iterations 9
Odds Ratios
Intercept                                                                                      0.010828
C(project_type_code)[T.Homelessness Prevention]                                               26.344496
C(project_type_code)[T.Other]                                                                  4.397959
C(project_type_code)[T.PH - Permanent Supportive Housing (disability required for entry)]     15.392857
C(project_type_code)[T.PH - Rapid Re-Housing]                                                128.319216
C(project_type_code)[T.Services Only]                                                         16.040977
C(project_type_code)[T.Street Outreach]                                                        1.924107
C(project_type_code)[T.Transitional housing]                                                   0.894934
dtype: float64
                            Logit Regre

In [ ]:
## too many iterations, > 35

# Logistic Regression for 'Organization Name'

# rename variable
logit_organization_name = smf.logit(formula='in_permanent_housing ~ C(organization_name)', 
                                    data=df_features).fit()
# odds ratios
print("Odds Ratios")
print(np.exp(logit_organization_name.params))
print(logit_organization_name.summary())

In [173]:
# Logistic Regression for 'Disability Type'

# rename variable
logit_disability = smf.logit(formula='in_permanent_housing ~ C(disability_type)', 
                             data=df_features).fit()
# odds ratios
print("Odds Ratios")
print(np.exp(logit_disability.params))
print(logit_disability.summary())

         Current function value: 0.321794
         Iterations: 35
Odds Ratios
Intercept                                            9.589041e-02
C(disability_type)[T.Both Alcohol and Drug Abuse]    8.197130e-01
C(disability_type)[T.Chronic Health Condition]       2.228687e+00
C(disability_type)[T.Developmental]                  9.480520e-01
C(disability_type)[T.Drug Abuse]                     1.642295e-01
C(disability_type)[T.HIV/AIDS]                       4.534162e-01
C(disability_type)[T.Mental Health Problem]          2.234694e+00
C(disability_type)[T.None]                           1.059616e+00
C(disability_type)[T.Physical]                       2.021866e+00
C(disability_type)[T.Substance Abuse]                4.487950e-07
dtype: float64
                            Logit Regression Results                            
Dep. Variable:     in_permanent_housing   No. Observations:                 3397
Model:                            Logit   Df Residuals:                     3387
Meth

/Users/annalie/anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [174]:
# Logistic Regression for 'Race'

# rename variable
logit_race = smf.logit(formula='in_permanent_housing ~ C(race)', 
                       data=df_features).fit()
# odds ratios
print("Odds Ratios")
print(np.exp(logit_race.params))
print(logit_race.summary())

         Current function value: 0.325747
         Iterations: 35
Odds Ratios
Intercept                                               7.211538e-02
C(race)[T.Asian]                                        4.078431e+00
C(race)[T.Black or African American]                    2.175163e+00
C(race)[T.Native Hawaiian or Other Pacific Islander]    3.555556e+00
C(race)[T.Other]                                        1.493025e-07
C(race)[T.Unknown]                                      2.718954e-01
C(race)[T.White]                                        1.495652e+00
dtype: float64
                            Logit Regression Results                            
Dep. Variable:     in_permanent_housing   No. Observations:                 3397
Model:                            Logit   Df Residuals:                     3390
Method:                             MLE   Df Model:                            6
Date:                  Tue, 15 Nov 2016   Pseudo R-squ.:                 0.01223
Time:              

/Users/annalie/anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [175]:
# Logistic Regression for 'Ethnicity'

# rename variable
logit_ethnicity = smf.logit(formula='in_permanent_housing ~ C(ethnicity)', 
                            data=df_features).fit()
# odds ratios
print("Odds Ratios")
print(np.exp(logit_ethnicity.params))
print(logit_ethnicity.summary())

Optimization terminated successfully.
         Current function value: 0.328072
         Iterations 8
Odds Ratios
Intercept                                  0.095921
C(ethnicity)[T.Non-Hispanic/Non-Latino]    1.358190
C(ethnicity)[T.Unknown]                    0.231671
dtype: float64
                            Logit Regression Results                            
Dep. Variable:     in_permanent_housing   No. Observations:                 3397
Model:                            Logit   Df Residuals:                     3394
Method:                             MLE   Df Model:                            2
Date:                  Tue, 15 Nov 2016   Pseudo R-squ.:                0.005177
Time:                          22:07:28   Log-Likelihood:                -1114.5
converged:                         True   LL-Null:                       -1120.3
                                          LLR p-value:                  0.003030
                                              coef    std err      

In [176]:
# Logistic Regression for 'Gender'

# rename variable
logit_gender = smf.logit(formula='in_permanent_housing ~ C(gender)', 
                            data=df_features).fit()
# odds ratios
print("Odds Ratios")
print(np.exp(logit_gender.params))
print(logit_gender.summary())

         Current function value: 0.323527
         Iterations: 35
Odds Ratios
Intercept                                  7.733492e-02
C(gender)[T.Male]                          2.053261e+00
C(gender)[T.Other]                         1.373076e-07
C(gender)[T.Transgender female to male]    2.032960e-08
C(gender)[T.Transgender male to female]    4.310256e+00
C(gender)[T.Unknown]                       6.956532e-07
dtype: float64
                            Logit Regression Results                            
Dep. Variable:     in_permanent_housing   No. Observations:                 3397
Model:                            Logit   Df Residuals:                     3391
Method:                             MLE   Df Model:                            5
Date:                  Tue, 15 Nov 2016   Pseudo R-squ.:                 0.01896
Time:                          22:09:13   Log-Likelihood:                -1099.0
converged:                        False   LL-Null:                       -1120.3
    

/Users/annalie/anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [191]:
# Logistic Regression for 'Chronic Homeless'

# rename variable
logit_chronic_homeless = smf.logit(formula='in_permanent_housing ~ C(chronic_homeless)', 
                                   data=df_features).fit()
# odds ratios
print("Odds Ratios")
print(np.exp(logit_chronic_homeless.params))
print(logit_chronic_homeless.summary())

Optimization terminated successfully.
         Current function value: 0.329429
         Iterations 6
Odds Ratios
Intercept                   0.110469
C(chronic_homeless)[T.1]    1.325514
dtype: float64
                            Logit Regression Results                            
Dep. Variable:     in_permanent_housing   No. Observations:                 3397
Model:                            Logit   Df Residuals:                     3395
Method:                             MLE   Df Model:                            1
Date:                  Tue, 15 Nov 2016   Pseudo R-squ.:                0.001062
Time:                          22:22:09   Log-Likelihood:                -1119.1
converged:                         True   LL-Null:                       -1120.3
                                          LLR p-value:                    0.1229
                               coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------

In [192]:
# Logistic Regression for 'Domestic Violence Victim'

# rename variable
logit_domestic_violence = smf.logit(formula='in_permanent_housing ~ C(domestic_violence_victim)', 
                                    data=df_features).fit()
# odds ratios
print("Odds Ratios")
print(np.exp(logit_domestic_violence.params))
print(logit_domestic_violence.summary())

Optimization terminated successfully.
         Current function value: 0.319367
         Iterations 7
Odds Ratios
Intercept                           0.146445
C(domestic_violence_victim)[T.1]    0.276045
dtype: float64
                            Logit Regression Results                            
Dep. Variable:     in_permanent_housing   No. Observations:                 3397
Model:                            Logit   Df Residuals:                     3395
Method:                             MLE   Df Model:                            1
Date:                  Tue, 15 Nov 2016   Pseudo R-squ.:                 0.03157
Time:                          22:24:47   Log-Likelihood:                -1084.9
converged:                         True   LL-Null:                       -1120.3
                                          LLR p-value:                 4.073e-17
                                       coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------

In [193]:
# Logistic Regression for 'Veteran Status'

# rename variable
logit_veteran = smf.logit(formula='in_permanent_housing ~ C(veteran_status)', 
                          data=df_features).fit()
# odds ratios
print("Odds Ratios")
print(np.exp(logit_veteran.params))
print(logit_veteran.summary())

Optimization terminated successfully.
         Current function value: 0.281517
         Iterations 7
Odds Ratios
Intercept                 0.054566
C(veteran_status)[T.1]    9.051720
dtype: float64
                            Logit Regression Results                            
Dep. Variable:     in_permanent_housing   No. Observations:                 3397
Model:                            Logit   Df Residuals:                     3395
Method:                             MLE   Df Model:                            1
Date:                  Tue, 15 Nov 2016   Pseudo R-squ.:                  0.1463
Time:                          22:24:57   Log-Likelihood:                -956.31
converged:                         True   LL-Null:                       -1120.3
                                          LLR p-value:                 2.768e-73
                             coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------

In [194]:
# Logistic Regression for 'Continuously Homeless One Year'

# rename variable
logit_continuously_homeless = smf.logit(formula='in_permanent_housing ~ C(continuously_homeless_one_year)', 
                                        data=df_features).fit()
# odds ratios
print("Odds Ratios")
print(np.exp(logit_continuously_homeless.params))
print(logit_continuously_homeless.summary())

Optimization terminated successfully.
         Current function value: 0.323351
         Iterations 6
Odds Ratios
Intercept                                 0.088699
C(continuously_homeless_one_year)[T.1]    2.222825
dtype: float64
                            Logit Regression Results                            
Dep. Variable:     in_permanent_housing   No. Observations:                 3397
Model:                            Logit   Df Residuals:                     3395
Method:                             MLE   Df Model:                            1
Date:                  Tue, 15 Nov 2016   Pseudo R-squ.:                 0.01949
Time:                          22:25:14   Log-Likelihood:                -1098.4
converged:                         True   LL-Null:                       -1120.3
                                          LLR p-value:                 3.880e-11
                                             coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------

# Further analysis

- compare people that have a positive outcome or negative outcome with each other, e.g. by plots